<h1><center> ENSF 519.01 Applied Data Scince </center></h1>
<h2><center> Final Project </center></h2>
<h3><center> Oscar Predictions from Movie Scripts </center></h3>
<p><center> Mark Ricalde, Paul Chen, Brian Pho, Harjee Johal <center></p>

# Phase 1

## Data Preprocessing

## Visualization and Exploratory Data Analysis

# Phase 2

## Discriminative Models

## Generative Models

## Clustering Models

## Machine Learning Models

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# https://www.tensorflow.org/guide/keras/overview

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
layers.SimpleRNN()
layers.GRU()
layers.LSTM()